In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select([
                     'review_id',
                     'star_rating',
                     'helpful_votes',
                     'total_votes',
                     'vine',
                     'verified_purchase',
])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [8]:
filtered_df = vine_df.filter('total_votes>=20')
filtered_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [36]:
from pyspark.sql.functions import col
filtered_df.withColumn('helpful_votes', col('helpful_votes').cast('Integer'))
filtered_df.withColumn('total_votes', col('total_votes').cast('Integer'))

TypeError: ignored

In [40]:
new_df = filtered_df.withColumn('vote_ratio', col('helpful_votes').cast('Integer') / col('total_votes').cast('Integer'))
new_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         vote_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N| 0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|                0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|               0.84|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|0.45161290322580644|
|R2FJ94555FZH32|          2|           55|         60|   N|                N| 0.9166666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y| 0.7846153846153846|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N| 0.8611111111111112|
| R6KTC1OPIOIIG|    

In [41]:
new_filtered_df = new_df.filter('vote_ratio>=0.5')
new_filtered_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|        vote_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|               0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|              0.84|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|0.9166666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|0.7846153846153846|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|0.8611111111111112|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|0.5588235294117647|
|R36O341WWXXKNP|          5|  

In [49]:
vine_paid = new_filtered_df.filter(new_filtered_df.vine == 'Y')
vine_paid.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|        vote_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|0.8888888888888888|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|               1.0|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|0.7916666666666666|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|0.8461538461538461|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|0.7692307692307693|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|0.9019607843137255|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|0.7857142857142857|
| RQ5WD90PUNBU9|          5|  

In [50]:
vine_unpaid = new_filtered_df.filter(new_filtered_df.vine == 'N')
vine_unpaid.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|        vote_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|               0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|              0.84|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|0.9166666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|0.7846153846153846|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|0.8611111111111112|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|0.5588235294117647|
|R36O341WWXXKNP|          5|  

## Vine Paid Calcs

In [54]:
vine_paid_total_review = vine_paid.count()
vine_paid_total_review

94

In [56]:
vine_paid_fivestar_df = vine_paid.filter(vine_paid.star_rating == 5)
vine_paid_fivestar_count = vine_paid_fivestar_df.count()
vine_paid_fivestar_count

48

In [57]:
vine_paid_fivestar_percen = vine_paid_fivestar_count / vine_paid_total_review
vine_paid_fivestar_percen

0.5106382978723404

## Vine Unpaid Calcs

In [58]:
vine_unpaid_total_review = vine_unpaid.count()
vine_unpaid_total_review

40471

In [59]:
vine_unpaid_fivestar_df = vine_unpaid.filter(vine_unpaid.star_rating == 5)
vine_unpaid_fivestar_count = vine_unpaid_fivestar_df.count()
vine_unpaid_fivestar_count

15663

In [60]:
vine_unpaid_fivestar_percen = vine_unpaid_fivestar_count / vine_unpaid_total_review
vine_unpaid_fivestar_percen

0.38701786464381904